In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10.3.1

In [2]:
import os

# === CẤU HÌNH ===
base_dir = "/kaggle/input/fixed-original-segment-dataset/dataset"  # hoặc thư mục chứa 'images/' và 'labels/'

def count_files(folder):
    return len([f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))])

def report_counts(split):
    img_dir = os.path.join(base_dir, "images", split)
    lbl_dir = os.path.join(base_dir, "labels", split)

    img_count = count_files(img_dir) if os.path.exists(img_dir) else 0
    lbl_count = count_files(lbl_dir) if os.path.exists(lbl_dir) else 0

    print(f"📂 {split.capitalize()}:")
    print(f"    Số ảnh   : {img_count}")
    print(f"    Số label : {lbl_count}")
    print(f"    Mismatch : {abs(img_count - lbl_count)}\n")

# === GỌI BÁO CÁO ===
print(" Kiểm tra số lượng ảnh và label:")
for split in ["train", "val"]:
    report_counts(split)


 Kiểm tra số lượng ảnh và label:
📂 Train:
    Số ảnh   : 1618
    Số label : 1618
    Mismatch : 0

📂 Val:
    Số ảnh   : 411
    Số label : 411
    Mismatch : 0



In [3]:
# Kiểm tra GPU
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


True
Tesla T4


In [4]:
data_yaml = """
path: /kaggle/input/fixed-original-segment-dataset/dataset
train: images/train
val: images/val
names:
  0: unbet
  1: betrivers
  2: fanduel
  3: betway
  4: caesars
  5: bally
  6: draftkings
  7: pointsbet
  8: bet365
  9: fanatics
  10: betparx
  11: betmgm
  12: gilariver
  13: casino
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml)


In [5]:
from ultralytics import YOLO

# Tải model yolov8l-seg.pt
model = YOLO("Model_yolo_base/yolov8s-seg.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 22.8M/22.8M [00:00<00:00, 187MB/s]


In [6]:

model.train(
    data="data.yaml",           # sử dụng file vừa tạo
    epochs=150,
    imgsz=640,
    optimizer="Adam",
    batch=8,
    device=0,
    name="yolov8s-seg-kaggle",
)

# Nén toàn bộ folder runs vào file zip
!zip -r /kaggle/working/runs_yolo.zip /kaggle/working/runs


Ultralytics 8.3.158 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=Model_yolo_base/yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s-seg-kaggle, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0,

100%|██████████| 755k/755k [00:00<00:00, 17.6MB/s]


Overriding model.yaml nc=80 with nc=14

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 77.8MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.2±0.3 ms, read: 39.1±16.8 MB/s, size: 289.9 KB)


...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   8%|▊         | 2/26 [00:00<00:08,  2.71it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:06<00:00,  4.02it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        411       3957      0.878      0.778      0.843      0.703      0.867      0.766      0.827      0.538
                 unbet        388       3340       0.92      0.814      0.909      0.695      0.872      0.769      0.839      0.439
             betrivers         13         17      0.968      0.765      0.879      0.713      0.969      0.765      0.877      0.513
               fanduel        161        277      0.949       0.87      0.951      0.776      0.913      0.837      0.914      0.485
                betway         60         91      0.929      0.858      0.915      0.773      0.929      0.858      0.918      0.584
               caesars          4         10      0.847      0.556       0.66      0.567      0.844      0.544       0.66      0.377
                 bally         52         56      0.931      0.893      0.952      0.721      0.876      0.839       0.83      0.301
            draftkings         49         59      0.962      0.864   

In [7]:
# Nén toàn bộ folder runs vào file zip
!zip -r /kaggle/working/runs_yolo.zip /kaggle/working/runs

updating: kaggle/working/runs/ (stored 0%)
updating: kaggle/working/runs/segment/ (stored 0%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/ (stored 0%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/train_batch0.jpg (deflated 6%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/val_batch2_pred.jpg (deflated 4%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/MaskPR_curve.png (deflated 9%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/val_batch2_labels.jpg (deflated 4%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/weights/ (stored 0%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/weights/last.pt (deflated 18%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/weights/best.pt (deflated 18%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/train_batch1.jpg (deflated 4%)
updating: kaggle/working/runs/segment/yolov8s-seg-kaggle/train_batch2.jpg (deflated 5%)
updating: kaggle/working/runs/segment/yolov8s-